# Simulation of Stepper Control methods

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
%matplotlib qt

### Input will be a frequency with a time period
stepFunc( frequency, time_to_stay): 
    - frequency: the next frequency (key) to which the gantry needs to move
    - time_to_stay: The time that the gantry must spend at the key (in seconds)
    
*** Maybe add volume control and key press duration to single function ***

In [2]:
class Gantry:
    def __init__(self, calibrate_k, const):
        self.calibration_key = calibrate_k
        self.previous_key = 0
        self.conversion_const = const # conversion const just 10 for now so that 10 steps = distance 
                                   # between keys
        
    def auto_home(self):
        print("Homing")
        print("Done")
        self.previous_key = self.calibration_key
        return self.calibration_key
        ## Homing function moves gantry until distance sensor reads out that the gantry 
        ## is at the end of the keyboard
#         Sudo code:
#             read current_position
#             if current_position == cal_key_position:
#                 end auto_home
#             else
#                 if current_position - cal_key_position < 0:
#                     dir = right
#                     move until current_position == cal_key_position
#                 else:
#                     dir = left
#                     move until current_position == cal_key_position
#             self.previous_key = cal_key

    def play_tune(self, freq_array, time_array, volume_array):
        keys = np.zeros_like(freq_array)
        vol_angles = np.zeros_like(volume_array)
        keys = np.rint(12 * np.log2(freq_array/440) + 36)
        vol_angles = 270*volume_array
        self.step_func(keys, time_array)
        return keys, vol_angles
    
    def step_func(self,key_list, time_to_stay): 
        for k,t in zip(key_list,time_to_stay):
            self.delta_k = self.previous_key - k
            if self.delta_k < 0 :
                print("DIR PIN SET TO LOW (MOVING RIGHT)")
                self.step_motor()
                time.sleep(t) # need to change sleep function to something else as it puts
                              # whole program to sleep (maybe a flag while servo is activated)
                print("Stayed for {} seconds".format(t))
            
            elif self.delta_k > 0:
                print("DIR PIN SET TO HIGH (MOVING LEFT)")
                self.step_motor()
                time.sleep(t)
                print("Stayed for {} seconds".format(t))
                
            else:
                time.sleep(t)
                print("Stayed for {} seconds".format(t))
                
            self.previous_key = k
    
    def step_motor(self):
        self.steps = int(np.abs(self.delta_k * self.conversion_const)) # Need to calculate what the conversion constant is
        print("Delta k: {} \t Steps: {} ".format(self.delta_k, self.steps))
        
        

In [ ]:
def dir_en(direction):
    global state
    if (direction == -1) & (state == 0):
        state = 1
        GPIO.output(DIR, GPIO.HIGH)
    elif (direction == -1) & (state ==1):
        state = 0
        GPIO.output(DIR, GPIO.LOW)
    return state
    

Since $ \Delta f $ between keys is not constant but changes to match the perception of pitch by the human ear, a transfer function from the linear frequency scale to the logarithmic key scale needs to be determined

In [76]:
key_freq = [55.0, 58.27047, 61.73541, 65.40639, 69.29566, 73.41619, 77.78175, 82.40689, 87.30706, 92.49861,
            97.99886, 103.8262, 110.0, 116.5409, 123.4708, 130.8128, 138.5913, 146.8324, 155.5635, 
            164.8138, 174.6141, 184.9972, 195.9977, 207.6523, 220.0, 233.0819, 246.9417, 261.6256, 
            277.1826, 293.6648, 311.127, 329.6276, 349.2282, 369.9944, 391.9954, 415.3047, 440.0, 
            466.1638, 493.8833, 523.2511, 554.3653, 587.3295, 622.254, 659.2551, 698.4565, 739.9888, 
            783.9909, 830.6094, 880.0, 932.3275, 987.7666, 1046.502, 1108.731, 1174.659, 1244.508]
keys = np.array([x for x in range(13,68)])
plt.plot(keys,(key_freq))
len(key_freq)

55

The frequency from note number equation is:

$f(n) = 2^{\frac{n-9}{12}} \times 440 Hz$

And so, conversely, the note number from the frequency is:

$n = 12 log_{2} (\frac{f}{440 Hz}) + 36$

*Note: The 49 constant is for a full sized keyboard

In [74]:
def key_num (frequency_list):
    keys = np.zeros_like(frequency_list)
    keys = 12 * np.log2(frequency_list/440) + 36
    return np.rint(keys)

In [85]:
k = key_num(np.array(key_freq))
print(k)

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53.
 54.]


In [3]:
g = Gantry(1, 10)
cal_key = g.auto_home()
print('Calibrated to key {} for home'.format(cal_key))

Homing
Done
Calibrated to key 1 for home


In [4]:
with open('frequency_list.csv','r') as file1:
    freq_test_arr = np.genfromtxt(file1, delimiter = ',')

with open('volume_list.csv','r') as file2:
    vol_test_arr = np.genfromtxt(file2, delimiter = ',')

time_arr = np.zeros_like(freq_test_arr)    

In [7]:
len(freq_test_arr)

536

In [5]:
keys, vols = g.play_tune(freq_test_arr, time_arr, vol_test_arr)

DIR PIN SET TO LOW (MOVING RIGHT)
Delta k: -46.0 	 Steps: 460 
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
Stayed for 0.0 seconds
DIR PIN SET TO LOW (MOVING RIGHT)
Delta k: -4.0 	 Steps: 40 
Stayed for 0.0 seconds
St

In [6]:
xAxis = np.array([0.  , 0.38, 0.75, 1.13, 1.5 , 1.88, 2.26, 2.63])
xticker = np.arange(len(keys))
fig, ax1 = plt.subplots()
fig.suptitle("Keys and volume of played notes")
ax1.set_ylabel("Key number ", color = "blue")
ax1.set_xlabel("Time ")
ax1.plot(keys, color = 'blue')

ax2 = ax1.twinx()
ax2.set_ylabel("Volume knob degrees", color = "orange")
ax2.plot(vols, color = 'orange')
plt.xticks(xticker[::66], xAxis)

([<matplotlib.axis.XTick at 0x20035652e80>,
 [Text(0, 0, '0.0'),
  Text(0, 0, '0.38'),
  Text(0, 0, '0.75'),
  Text(0, 0, '1.13'),
  Text(0, 0, '1.5'),
  Text(0, 0, '1.88'),
  Text(0, 0, '2.26'),
  Text(0, 0, '2.63')])